In [ ]:
import os, csv, pickle
from xml.dom import minidom
from xml.etree import ElementTree as ET
from collections import defaultdict
import re
from tqdm import tqdm

from utils import  walkData

In [ ]:
def get_path_of_all_xml_file():
	input_file = "../data/trials/all_xml.txt"
	with open(input_file, 'r') as fin:
		lines = fin.readlines()
	input_file_lst = [i.strip() for i in lines]
	return input_file_lst## 优化text_encoder
## 优化text_encoder
input_file_lst = get_path_of_all_xml_file()## 优化text_encoder
print(len(input_file_lst),input_file_lst[:10])

In [ ]:
def tags2key(tags: list):
    return '_'.join(tags)


def xml_file2text(xml_file,my_list):
    tree = ET.parse(xml_file)

    result = []
    for tags in my_list:
        root = tree.getroot()
        for tag in tags:
            if root:
                root = root.find(tag)
            else:
                break
        try:
            result.append(root.text)
        except:
            result.append("")

    return result

def find_text(my_list, LEN=len(input_file_lst)):
    result_dict = {}
    key_list=[]
    if len(my_list)==0 or LEN<=0:
        return
    if LEN > len(input_file_lst):
        LEN=len(input_file_lst)
    for tags in my_list:
        key=tags2key(tags)
        result_dict[key]=[]
        key_list.append(key)
    for file in tqdm(input_file_lst[:LEN]):
        result = xml_file2text(f"../data/{file}",my_list)
        for i in range(len(my_list)):
            result_dict[key_list[i]].append(result[i])
    return result_dict


In [ ]:
my_list=[['id_info', 'nct_id'],['study_type'],['brief_title'],['official_title'],['oversight_info', 'has_dmc'],
         ['brief_summary', 'textblock'], ['detailed_description', 'textblock'], ['overall_status'], ['start_date'], ['completion_date'],[ 'phase'], ['study_design_info', 'observational_model'], ['study_design_info', 'time_perspective'],['study_design_info' ,'intervention_model'],['study_design_info' ,'primary_purpose'],['study_design_info' ,'masking'],['condition'],['primary_outcome', 'time_frame'],['number_of_groups'],['enrollment'],['location_countries', 'country']]
my_dict = find_text(my_list, 10000)
key_list=list(map(tags2key, my_list))
print(my_dict.keys())
print(key_list)

In [ ]:
def lazy_find(i):
    print(key_list[i], set(my_dict[key_list[i]]))

lazy_find(-3)

# my_dict['study_type']

In [ ]:
# my_dict['brief_title']
'0'+'1'

In [ ]:
def xml_file2tag(xml_file,my_list):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    study_type = root.find('study_type').text
    if study_type != 'Interventional':
        return ''
    interventions = [i for i in root.findall('intervention')]
    drug_interventions = [i.find('intervention_name').text for i in interventions \
														if i.find('intervention_type').text=='Drug']
														# or i.find('intervention_type').text=='Biological']
    if len(drug_interventions)==0:
        return ''
    result = []
    for tags in my_list:
        root = tree.getroot()
        for tag in tags:
            # print(tag)
            if root:
                root = root.find(tag)
            else:
                break
        try:
            # result.append(root.text)
            child_list=[child.tag for child in root]
            # for child in root:
            #     child_list.append(child)
            result.append(child_list)
        except:
            result.append("")

    return result

def find_tag(my_list, LEN=len(input_file_lst)):
    result_dict = {}
    key_list=[]
    if len(my_list)==0 or LEN<=0:
        return
    if LEN > len(input_file_lst):
        LEN=len(input_file_lst)
    for tags in my_list:
        key=tags2key(tags)
        result_dict[key]=[]
        key_list.append(key)
    for file in tqdm(input_file_lst[:LEN]):
        result = xml_file2tag(f"../data/{file}",my_list)
        if result=='':
            continue
        for i in range(len(my_list)):
            result_dict[key_list[i]].append(result[i])
    return result_dict


In [ ]:
tag_list=[['study_design_info'], ['intervention'], ['arm_group'],['secondary_outcome']]
tag_dict = find_tag(tag_list, 10000)
tagkey_list=list(map(tags2key, tag_list))
print(tag_dict.keys())
print(tagkey_list)

In [ ]:
def flatten(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):  # 如果是子列表，递归调用 flatten 函数
            flat_list.extend(flatten(item))
        else:
            flat_list.append(item)  # 否则直接添加到结果列表
    return flat_list

def taglazy_find(i):
    print(tagkey_list[i], set(flatten(tag_dict[tagkey_list[i]])))

taglazy_find(0)
# my_dict['study_type']
# flattened_list = flatten(tag_dict['study_design_info'])
# print(set(flattened_list))

In [ ]:
def xml_file2alltext(xml_file,my_list):
    tree = ET.parse(xml_file)
    result = []
    for tags in my_list:
        root = tree.getroot()
        cnt=0
        outcome=[]
        elements=[]
        root = root.findall(tags[0])
        for tag in tags[1:]:
            if root:
                for e in root:
                    e = e.findall(tag)
                    elements.extend(e)
                root = elements
                elements=[]
            else:
                break
        try:
            for e in root:
                cnt+=1
                outcome.append(e.text)
            result.append([cnt]+outcome)
        except:
            result.append([0, ''])

    return result

def find_alltext(my_list, LEN=len(input_file_lst)):
    result_dict = {}
    key_list=[]
    if len(my_list)==0 or LEN<=0:
        return
    if LEN > len(input_file_lst):
        LEN=len(input_file_lst)
    for tags in my_list:
        key=tags2key(tags)
        result_dict[key]=[]
        key_list.append(key)
    for file in tqdm(input_file_lst[:LEN]):
        result = xml_file2alltext(f"../data/{file}",my_list)
        for i in range(len(my_list)):
            result_dict[key_list[i]].append(result[i])
    return result_dict



In [ ]:
cnt_list=[['id_info', 'nct_id'],['primary_outcome', 'time_frame'],['secondary_outcome','time_frame'],['primary_outcome'],['secondary_outcome'], ['intervention'], ['condition'], ['location'], ['arm_group'],['condition_browse','mesh_term'],['intervention_browse','mesh_term']]
cnt_dict = find_alltext(cnt_list,10000)
cntkey_list=list(map(tags2key, cnt_list))
print(cnt_dict.keys())
print(cntkey_list)

In [ ]:
def alltextlazy_find(i,LEN):
    print(cntkey_list[i], set(flatten(cnt_dict[cntkey_list[i]][:LEN])))
alltextlazy_find(3,10000)
set(flatten(cnt_dict['secondary_outcome_time_frame']))
# for idx, e in enumerate(cnt_dict['primary_outcome_time_frame']):
#     if e[0] == 20:
#         print(f"Index: {idx}, Value: {e}")
#         break


In [ ]:
def flatten(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):  # 如果是子列表，递归调用 flatten 函数
            flat_list.extend(flatten(item))
        else:
            flat_list.append(item)  # 否则直接添加到结果列表
    return flat_list
# cnt_dict['id_info_nct_id'][8125]
times = list(set(flatten(cnt_dict['secondary_outcome_time_frame'])))


In [ ]:
word_to_num = [
    ('one', '1'),
    ('two', '2'),
    ('three', '3'),
    ('four', '4'),
    ('five', '5'),
    ('six', '6'),
    ('seven', '7'),
    ('eight', '8'),
    ('nine', '9'),
    ('ten', '10')
]
wd_2_word = [
    (' yr ', ' year '),
    (' yrs ', ' years '),
    (' mo ', ' month '),
    (' mos ', ' months '),
    (' d ', ' day '),
    (' ds ', ' days '),
    (' wk ', ' week '),
    (' wks ', ' weeks ')
]
def math_process(time):
    # 英文2数字，处理简写,逗号式，合并相邻数字
    for k,v in word_to_num:
        time.replace(k,v)
    for k,v in wd_2_word:
        time.replace(k,v)
    # time = re.sub(r'(\d),(\d)', r'\1\2', time)
    time = time.replace(',', '')
    match = re.search(r"(\d+)\s(\d+)", time)
    while match:
        num = str(max(int(match.group(1)),int(match.group(2))))
        time = time.replace(match.group(),num)
        match = re.search(r"(\d+)\s(\d+)", time)
    match = re.search(r"(\d+)\sand\s(\d+)", time)
    while match:
        num = str(max(int(match.group(1)),int(match.group(2))))
        time = time.replace(match.group(),num)
        match = re.search(r"(\d+)\sand\s(\d+)", time)
    return time

def get_time(times):
    # 合并数值， 处理数值
    # 数值匹配
    # 单位统一并选出最大值
    for t in times:
        t = math_process(str(t).lower())
        print(t)
    pass
get_time(times[:])
# print(math_process('up to Day 9, 30, 90 and 180'))

In [ ]:
# print(set(flatten(cnt_dict['tertiary_outcome'])))
# print(set(flatten(cnt_dict['primary_outcome'])))
# print(list({''}))
list(set([]))

In [ ]:
def xml_file2conditions(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    conditions = [i.text for i in root.findall('condition')]
    conditions = [i.lower() for i in conditions]

    return conditions

def print_conditions(LEN=len(input_file_lst)):
    # result_dict = {}
    for file in tqdm(input_file_lst[:LEN]):
        result = xml_file2conditions(f"../data/{file}")
        print(result)
    return

print_conditions(100)

## 优化text_encoder

In [4]:
import pandas as pd
import torch
import pickle
data = pd.read_csv(f'../data/time_prediction_input.csv', sep='\t', dtype={'masking': str, 'intervention_model': str})
sentence_2_vec = pickle.load(open('../data/time_frame2embedding.pkl', 'rb'))

In [7]:
sentence_2_vec = pickle.load(open('../data/time_frame2embedding.pkl', 'rb'))

wd_2_word = [
    (' yr ', ' year '),
    (' yrs ', ' years '),
    (' mo ', ' month '),
    (' mos ', ' months '),
    (' d ', ' day '),
    (' ds ', ' days '),
    (' wk ', ' week '),
    (' wks ', ' weeks ')
]
from tqdm import tqdm
def clean_text(text):
    if pd.isnull(text):
        return None
    text = text.lower()
    for k,v in wd_2_word:
        text.replace(k,v)
    text_split = text.split('\n')
    filter_out_empty_fn = lambda x: len(x.strip())>0
    strip_fn = lambda x:x.strip()
    text_split = list(filter(filter_out_empty_fn, text_split))
    text_split = list(map(strip_fn, text_split))
    text = ''.join(text_split)
    return text

def time_frame2embedding(text_lst):
    text_emb=[]
    for texts in tqdm(text_lst):
        texts = eval(texts)
        for i in range(len(texts)):
            texts[i] = clean_text(texts[i])
        if not texts:
            # print("Warning: Empty text is found")
            text_emb.append(torch.zeros(768, dtype=torch.float32))
        else:
            try:
                texts_emb, _ = torch.max(
                    torch.stack([sentence_2_vec[text] for text in texts]), dim=0)
                text_emb.append(texts_emb)
                # text_emb.append(sentence_2_vec[text])
            except:
                print("Warning: Error2 text, not emb time_frame:", texts)
                text_emb.append(torch.zeros(768, dtype=torch.float32))
                exit()
    return torch.stack(text_emb)  # len(text_list), 768

# title_emb = text2embedding(data['title'].tolist())
# summary_emb = text2embedding(data['summary'].tolist())
print("start")
# detail_emb = text2embedding(data['detail'].tolist())
# print(data['time_frame'].tolist())
time_frame_emb = time_frame2embedding(data['time_frame'].tolist())

start


100%|██████████| 3800/3800 [00:00<00:00, 28494.83it/s]


In [3]:
import pickle
sentence_2_vec = pickle.load(open('../data/time_frame2embedding.pkl', 'rb'))
texts = ['90 days']
[sentence_2_vec[text] for text in texts]

[tensor([-1.2656e-01,  7.9278e-03, -3.1774e-01,  8.3786e-02, -2.3258e-02,
         -1.7356e-01,  2.8440e-01,  5.7807e-01, -4.6646e-01, -1.7170e-01,
          1.2471e-01,  5.4870e-02,  1.8617e-01,  2.5144e-01,  1.4728e-01,
          2.1588e-01, -4.9575e-01, -5.0732e-03,  3.9103e-01, -6.0163e-01,
          1.9910e-01, -1.0090e-01,  3.4842e-03, -4.9043e-02,  5.3031e-02,
          9.2863e-02, -1.4985e-01,  7.7242e-02,  1.3294e-01, -4.8438e-03,
         -3.4925e-01,  3.0071e-02, -3.0298e-01,  1.0916e-01,  5.4432e-01,
          1.2725e-01, -9.9560e-02, -1.7259e-02, -2.4105e-02, -1.1222e-01,
          7.5558e-02, -5.1295e-02,  1.1221e-01, -5.9356e-01,  1.3076e-01,
         -2.3687e-01, -1.7795e+00, -2.3276e-01, -7.7510e-02, -6.3919e-02,
          1.7728e-01, -7.3679e-02,  8.9259e-01,  5.0958e-02, -5.0525e-03,
          4.6914e-01, -1.2154e-01,  5.6035e-01, -2.0627e-01,  5.4802e-01,
         -1.6342e-01, -1.9803e-01, -2.5869e-01,  1.7896e-01, -2.7591e-02,
          4.1934e-01,  4.4480e-02,  3.